# MRecomender 

#### Recomendador de Peliculas, el dataset fue sacado de kaggle, 

### Load Dataset

In [1]:
import pandas as pd

df = pd.read_csv("movies_metadata.csv")
df

C:\Users\luigu\AppData\Local\Temp\ipykernel_10096\3796868268.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


### Convert row genres from dictionary to list

In [2]:
import ast


def get_genres_list(genres_str):
    if pd.isna(genres_str) or genres_str == '[]':
        return []
    return [g['name'] for g in ast.literal_eval(genres_str)]


df['genres_list'] = df['genres'].apply(get_genres_list)


all_genres = sorted(list(set([genre for sublist in df['genres_list'] for genre in sublist])))
print(all_genres)

['Action', 'Adventure', 'Animation', 'Aniplex', 'BROSTA TV', 'Carousel Productions', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 'GoHands', 'History', 'Horror', 'Mardock Scramble Production Committee', 'Music', 'Mystery', 'Odyssey Media', 'Pulser Productions', 'Rogue State', 'Romance', 'Science Fiction', 'Sentai Filmworks', 'TV Movie', 'Telescene Film Group Productions', 'The Cartel', 'Thriller', 'Vision View Entertainment', 'War', 'Western']


### We keep only rows relevant to KNN

In [3]:
df_new = df[['title', 'vote_average', 'vote_count', 'runtime', 'budget', 'revenue', 'genres_list']]

df_new['genres_str'] = df_new['genres_list'].apply(lambda x: ', '.join(x))


print(df_new.head())

                         title  vote_average  vote_count  runtime    budget  \
0                    Toy Story           7.7      5415.0     81.0  30000000   
1                      Jumanji           6.9      2413.0    104.0  65000000   
2             Grumpier Old Men           6.5        92.0    101.0         0   
3            Waiting to Exhale           6.1        34.0    127.0  16000000   
4  Father of the Bride Part II           5.7       173.0    106.0         0   

       revenue                   genres_list                  genres_str  
0  373554033.0   [Animation, Comedy, Family]   Animation, Comedy, Family  
1  262797249.0  [Adventure, Fantasy, Family]  Adventure, Fantasy, Family  
2          0.0             [Romance, Comedy]             Romance, Comedy  
3   81452156.0      [Comedy, Drama, Romance]      Comedy, Drama, Romance  
4   76578911.0                      [Comedy]                      Comedy  


C:\Users\luigu\AppData\Local\Temp\ipykernel_10096\1610536819.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['genres_str'] = df_new['genres_list'].apply(lambda x: ', '.join(x))


In [4]:
type(all_genres)

def genres_norm(l):
    temp = [0]*len(all_genres)
    for x in range(len(all_genres)):
        if all_genres[x] in l:
            temp[x] = 1
    return temp

In [5]:
df_new['genres_vector'] = df_new['genres_list'].apply(genres_norm)
df_new

C:\Users\luigu\AppData\Local\Temp\ipykernel_10096\2678734919.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['genres_vector'] = df_new['genres_list'].apply(genres_norm)


,title,vote_average,vote_count,runtime,budget,revenue,genres_list,genres_str,genres_vector
0,Toy Story,7.7,5415.0,81.0,30000000,373554033.0,"[Animation, Comedy, Family]","Animation, Comedy, Family","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,Jumanji,6.9,2413.0,104.0,65000000,262797249.0,"[Adventure, Fantasy, Family]","Adventure, Fantasy, Family","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
2,Grumpier Old Men,6.5,92.0,101.0,0,0.0,"[Romance, Comedy]","Romance, Comedy","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Waiting to Exhale,6.1,34.0,127.0,16000000,81452156.0,"[Comedy, Drama, Romance]","Comedy, Drama, Romance","[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ..."
4,Father of the Bride Part II,5.7,173.0,106.0,0,76578911.0,[Comedy],Comedy,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
45461,Subdue,4.0,1.0,90.0,0,0.0,"[Drama, Family]","Drama, Family","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."
45462,Century of Birthing,9.0,3.0,360.0,0,0.0,[Drama],Drama,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
45463,Betrayal,3.8,6.0,90.0,0,0.0,"[Action, Drama, Thriller]","Action, Drama, Thriller","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
45464,Satan Triumphant,0.0,0.0,87.0,0,0.0,[],,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
df_new['budget'] = pd.to_numeric(df_new['budget'], errors='coerce')
df_new['budget'] = df_new['budget'].astype(float)

C:\Users\luigu\AppData\Local\Temp\ipykernel_10096\6257630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['budget'] = pd.to_numeric(df_new['budget'], errors='coerce')
C:\Users\luigu\AppData\Local\Temp\ipykernel_10096\6257630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['budget'] = df_new['budget'].astype(float)


In [7]:
df_new = df_new.drop(['genres_list', 'genres_str'],axis =1)
df_new

,title,vote_average,vote_count,runtime,budget,revenue,genres_vector
0,Toy Story,7.7,5415.0,81.0,30000000.0,373554033.0,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,Jumanji,6.9,2413.0,104.0,65000000.0,262797249.0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
2,Grumpier Old Men,6.5,92.0,101.0,0.0,0.0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Waiting to Exhale,6.1,34.0,127.0,16000000.0,81452156.0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ..."
4,Father of the Bride Part II,5.7,173.0,106.0,0.0,76578911.0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
45461,Subdue,4.0,1.0,90.0,0.0,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ..."
45462,Century of Birthing,9.0,3.0,360.0,0.0,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
45463,Betrayal,3.8,6.0,90.0,0.0,0.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
45464,Satan Triumphant,0.0,0.0,87.0,0.0,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
import numpy as np

genres_matrix = np.array(df_new['genres_vector'].tolist())

numeric_features = df_new[['vote_average', 'vote_count', 'runtime', 'budget', 'revenue']].values
all_features = np.hstack([numeric_features, genres_matrix])
print(all_features[0])

[7.70000000e+00 5.41500000e+03 8.10000000e+01 3.00000000e+07
 3.73554033e+08 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00]


In [16]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('knn', NearestNeighbors(n_neighbors=5, metric='euclidean'))
])

pipeline.fit(all_features)
distances, indices = pipeline['knn'].kneighbors([pipeline['scaler'].transform([pipeline['imputer'].transform(all_features)[0]])[0]])

print("Indices vecinos:", indices)
print("Distancias:", distances)

Indices vecinos: [[    0 15472  2997  5084  4756]]
Distancias: [[2.38418579e-07 4.78002276e+00 5.00240927e+00 5.08162488e+00
  5.89812599e+00]]


In [20]:
for x in indices:
    print(df_new['title'][x])

0             Toy Story
15472     Despicable Me
2997        Toy Story 2
5084            Ice Age
4756     Monsters, Inc.
Name: title, dtype: object
